In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score

# Define U-Net Model
class UNet(nn.Module):
    # Define the U-Net architecture here
    # For brevity, this is a placeholder
    def __init__(self):
        super(UNet, self).__init__()
        # Define layers here
    
    def forward(self, x):
        # Forward pass implementation here
        return x

# Define FPN Model
class FPN(nn.Module):
    # Define the FPN architecture here
    # For brevity, this is a placeholder
    def __init__(self):
        super(FPN, self).__init__()
        # Define layers here
    
    def forward(self, x):
        # Forward pass implementation here
        return x

# Define Siamese Network Model
class SiameseNetwork(nn.Module):
    # Define the Siamese Network architecture here
    # For brevity, this is a placeholder
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Define layers here
    
    def forward(self, x1, x2):
        # Forward pass implementation here
        return x1, x2

# Define ResNet-based Model
class ResNetChangeDetector(nn.Module):
    def __init__(self):
        super(ResNetChangeDetector, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 2)  # Adjust output layer for change detection
    
    def forward(self, x):
        return self.resnet(x)

# Dataset loading and preprocessing
class ChangeDetectionDataset(Dataset):
    def __init__(self, image_paths, label_paths, transform=None):
        self.image_paths = image_paths
        self.label_paths = label_paths
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        label = Image.open(self.label_paths[idx])
        
        if self.transform:
            image = self.transform(image)
            label = self.transform(label)
        
        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Create dataset and dataloader
train_dataset = ChangeDetectionDataset(image_paths=['path/to/images'], label_paths=['path/to/labels'], transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Initialize models
unet = UNet()
fpn = FPN()
siamese_net = SiameseNetwork()
resnet = ResNetChangeDetector()

# Define loss function and optimizers
criterion = nn.BCEWithLogitsLoss()
optimizer_unet = optim.Adam(unet.parameters(), lr=1e-4)
optimizer_fpn = optim.Adam(fpn.parameters(), lr=1e-4)
optimizer_siamese = optim.Adam(siamese_net.parameters(), lr=1e-4)
optimizer_resnet = optim.Adam(resnet.parameters(), lr=1e-4)

# Training loop (for one epoch as an example)
def train_model(model, optimizer, criterion, dataloader):
    model.train()
    running_loss = 0.0
    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    return running_loss / len(dataloader.dataset)

# Training the models
loss_unet = train_model(unet, optimizer_unet, criterion, train_loader)
loss_fpn = train_model(fpn, optimizer_fpn, criterion, train_loader)
loss_siamese = train_model(siamese_net, optimizer_siamese, criterion, train_loader)
loss_resnet = train_model(resnet, optimizer_resnet, criterion, train_loader)

print(f'UNet Loss: {loss_unet:.4f}')
print(f'FPN Loss: {loss_fpn:.4f}')
print(f'Siamese Network Loss: {loss_siamese:.4f}')
print(f'ResNet Loss: {loss_resnet:.4f}')

# Evaluation and Metrics
def evaluate_model(model, dataloader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            preds = torch.sigmoid(outputs).cpu().numpy()
            labels = labels.cpu().numpy()
            y_true.extend(labels.flatten())
            y_pred.extend(preds.flatten())
    return precision_score(y_true, y_pred), recall_score(y_true, y_pred), f1_score(y_true, y_pred), jaccard_score(y_true, y_pred)

# Evaluate the models
precision_unet, recall_unet, f1_unet, iou_unet = evaluate_model(unet, train_loader)
precision_fpn, recall_fpn, f1_fpn, iou_fpn = evaluate_model(fpn, train_loader)
precision_siamese, recall_siamese, f1_siamese, iou_siamese = evaluate_model(siamese_net, train_loader)
precision_resnet, recall_resnet, f1_resnet, iou_resnet = evaluate_model(resnet, train_loader)

print(f'UNet Precision: {precision_unet:.4f}, Recall: {recall_unet:.4f}, F1: {f1_unet:.4f}, IoU: {iou_unet:.4f}')
print(f'FPN Precision: {precision_fpn:.4f}, Recall: {recall_fpn:.4f}, F1: {f1_fpn:.4f}, IoU: {iou_fpn:.4f}')
print(f'Siamese Network Precision: {precision_siamese:.4f}, Recall: {recall_siamese:.4f}, F1: {f1_siamese:.4f}, IoU: {iou_siamese:.4f}')
print(f'ResNet Precision: {precision_resnet:.4f}, Recall: {recall_resnet:.4f}, F1: {f1_resnet:.4f}, IoU: {iou_resnet:.4f}')

# Visualization
def visualize_results(images, labels, preds):
    plt.figure(figsize=(12, 8))
    for i in range(len(images)):
        plt.subplot(3, len(images), i + 1)
        plt.imshow(images[i].permute(1, 2, 0))
        plt.title('Input Image')
        plt.axis('off')
        
        plt.subplot(3, len(images), len(images) + i + 1)
        plt.imshow(labels[i].squeeze(), cmap='gray')
        plt.title('Ground Truth')
        plt.axis('off')
        
        plt.subplot(3, len(images), 2 * len(images) + i + 1)
        plt.imshow(preds[i].squeeze(), cmap='gray')
        plt.title('Prediction')
        plt.axis('off')
    
    plt.show()

# Visualize results for a few examples
images, labels = next(iter(train_loader))
preds_unet = torch.sigmoid(unet(images)).cpu().numpy()
preds_fpn = torch.sigmoid(fpn(images)).cpu().numpy()
preds_siamese = torch.sigmoid(siamese_net(images, images)).cpu().numpy()
preds_resnet = torch.sigmoid(resnet(images)).cpu().numpy()

visualize_results(images, labels, preds_unet)
visualize_results(images, labels, preds_fpn)
visualize_results(images, labels, preds_siamese)
visualize_results(images, labels, preds_resnet)


/home/amehmood/anaconda3/envs/main_env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/amehmood/anaconda3/envs/main_env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ValueError: optimizer got an empty parameter list

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score

# Define U-Net Model
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 1, kernel_size=2, stride=2),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Define FPN Model
class FPN(nn.Module):
    def __init__(self):
        super(FPN, self).__init__()
        self.backbone = models.resnet50(pretrained=True)
        self.backbone.fc = nn.Identity()  # Remove the classification layer
        self.lateral = nn.Conv2d(2048, 256, kernel_size=1)
        self.smooth = nn.Conv2d(256, 1, kernel_size=1)
    
    def forward(self, x):
        x = self.backbone(x)
        x = self.lateral(x)
        x = self.smooth(x)
        return x

# Define Siamese Network Model
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Sequential(
            nn.Linear(128*53*53, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 2)
        )
    
    def forward_one(self, x):
        x = self.cnn(x)
        x = x.view(x.size()[0], -1)
        x = self.fc(x)
        return x
    
    def forward(self, x1, x2):
        output1 = self.forward_one(x1)
        output2 = self.forward_one(x2)
        return output1, output2

# Define ResNet-based Model
class ResNetChangeDetector(nn.Module):
    def __init__(self):
        super(ResNetChangeDetector, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 1)  # Adjust output layer for change detection
    
    def forward(self, x):
        return self.resnet(x)

# Dataset loading and preprocessing
class ChangeDetectionDataset(Dataset):
    def __init__(self, image_paths, label_paths, transform=None):
        self.image_paths = image_paths
        self.label_paths = label_paths
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        label = Image.open(self.label_paths[idx])
        
        if self.transform:
            image = self.transform(image)
            label = self.transform(label)
        
        return image, l
